In [1]:
class IntegerValue:
    def __set__(self, instance, value):
        # try to hold the value inside the instance in the hardcodeed, dedicated variable
        instance.stored_value = int(value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return getattr(instance, "stored_value", None)


class Point1D:
    x = IntegerValue()


p1, p2 = Point1D(), Point1D()

p1.x = 10.1
p2.x = 20.1

p1.x, p2.x  # seems to work, but... `stored_value` is used - what if __slots__ are defined and it's not possible to use __dict__

(10, 20)

In [2]:
p1.__dict__, p2.__dict__

({'stored_value': 10}, {'stored_value': 20})

In [3]:
# this makes things even worse - `stored_value` from descriptor is going to be overwritten ....
class Point2D:
    x = IntegerValue()
    y = IntegerValue()


In [4]:
p1 = Point2D()
p1.x = 20
p1.y = 40
p1.__dict__, p1.x, p1.y  # x and y share the same variable for holding the value, also different instances of Point2D will do the same

({'stored_value': 40}, 40, 40)

In [5]:
class IntegerValue:
    def __init__(self, property_name):
        # what if this property name already existis in the class?
        # this would overwrite previous value
        self.storage_name =  "_" + property_name
    
    def __set__(self, instance, value):
        # try to hold the value inside the instance in the hardcodeed, dedicated variable
        setattr(instance, self.storage_name, value)

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return getattr(instance, self.storage_name, None)


In [6]:
class Point2D:
    # makes the job done  more or less, but it's ugly to use and it's easy to make an error
    x = IntegerValue("x")
    y = IntegerValue("y")

In [7]:
a = Point2D()
b = Point2D()

a.x = 10
a.y = 15

b.x = 1
b.y = 5

In [8]:
a.x, a.y, a.__dict__

(10, 15, {'_x': 10, '_y': 15})

In [9]:
b.x, b.y, b.__dict__

(1, 5, {'_x': 1, '_y': 5})

In [18]:
class IntegerValue:
    # another approach to the instance values holding problem
    # works on first sight, but it requires instance to be hashable and 
    # introduces memory leak as a bonus! :) 
    # Notice that instance has now additional reference - self.values is holding it,
    # that way garbage collector will never clean the instance memory
    def __init__(self):
        # each descriptor instance has it's own values dict
        self.values = {}

    def __set__(self, instance, value):
        # assuming instance is hashable!
        print(f"SETTING: {self} - {instance} - {value}")
        self.values[instance] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance)


In [29]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

a = Point2D()

In [30]:
a.__dict__

{}

In [31]:
a.x = 10
a.y = 20
a.x, a.y  # each property (x or y) has it's own descriptor instance which in turn has it's own dict `values` instance

SETTING: <__main__.IntegerValue object at 0x10bc70470> - <__main__.Point2D object at 0x10bc72660> - 10
SETTING: <__main__.IntegerValue object at 0x10bc705f0> - <__main__.Point2D object at 0x10bc72660> - 20


(10, 20)

In [32]:
Point2D.x.values

{<__main__.Point2D at 0x10bc72660>: 10}

In [33]:
b = Point2D()
b.x

In [34]:
b.x = 44

SETTING: <__main__.IntegerValue object at 0x10bc70470> - <__main__.Point2D object at 0x10bb8f650> - 44


In [35]:
Point2D.x.values, Point2D.y.values  # access to each instance values

({<__main__.Point2D at 0x10bc72660>: 10,
  <__main__.Point2D at 0x10bb8f650>: 44},
 {<__main__.Point2D at 0x10bc72660>: 20})

In [36]:
print(hex(id(a)))
del a  # won't clear the memorty, reference is still present inside data descriptors values!

0x10bc72660


In [38]:
Point2D.x.values  # object `a` still alive, well and kicking

{<__main__.Point2D at 0x10bc72660>: 10, <__main__.Point2D at 0x10bb8f650>: 44}

In [42]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [43]:
p1 = Point2D()
id_p1 = id(p1)

ref_count(id_p1)

1

In [46]:
p1.x = 1234
ref_count(id_p1)  # reference count incremented

SETTING: <__main__.IntegerValue object at 0x10bc70470> - <__main__.Point2D object at 0x10bcd8530> - 1234


2

In [47]:
"p1" in globals()


True

In [48]:
del p1

In [51]:
"p1" in globals()  # gives False, so we deleted the `p1` object?

False

In [52]:
# nope, still some references left
re